## Reqs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/NeuralClassifier/ReusabilityRate_ML
!git checkout corr_analysis
%cd /content/ReusabilityRate_ML/DataSets

Cloning into 'ReusabilityRate_ML'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 80 (delta 26), reused 7 (delta 2), pack-reused 0
Unpacking objects: 100% (80/80), done.
fatal: not a git repository (or any of the parent directories): .git
/content/ReusabilityRate_ML/DataSets


In [3]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statistics
import tensorflow as tf

import warnings
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Read class and package metrics(x:inputs) and RR(y:label)

In [4]:
cls_df = pd.read_excel('classes.xlsx')
pkg_df = pd.read_excel('packages.xlsx')
cls_df.drop(['Unnamed: 0'], axis=1, inplace=True)
pkg_df.drop(['Unnamed: 0'], axis=1, inplace=True)

## Class DNN

### Load data

In [10]:
target_scaler = MinMaxScaler(feature_range=(-10, 10))
data = cls_df
df = data.iloc[:,3:-1]
data_columns = df.columns
X_np = np.array(df)
target_scaler.fit(X_np)
X = X_np
#X = target_scaler.transform(X_np)

Y = data.iloc[:,-1]
Y = Y.values.reshape(-1, 1)
target_scaler = MinMaxScaler(feature_range=(-10, 10))
target_scaler.fit(Y)
#Y = target_scaler.transform(Y)

MinMaxScaler(feature_range=(-10, 10))

### Training data preparation 

In [11]:
#indexes = np.arange(len(X))
splitter = int(len(X) * .8)
X_train = X[:splitter]
Y_train = Y[:splitter]
X_val = X[splitter:]
Y_val = Y[splitter:]

### Train model

In [30]:
##model


def my_model_fully_connected(loss_fn, opt, ishape = 27, lr = 1e-2):
    model_input = tf.keras.layers.Input(shape = ishape)
    l2 = tf.keras.layers.Dense(100, activation = 'relu')(model_input)
    l2_drop = tf.keras.layers.Dropout(0.05)(l2)
    l3 = tf.keras.layers.Dense(50, activation = 'relu')(l2_drop)
    l4 = tf.keras.layers.Dense(25, activation = 'relu')(l3) 
    out= tf.keras.layers.Dense(1, activation = 'relu')(l4)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    model.compile(optimizer=opt, loss=loss_fn, metrics = ["accuracy"])
    return model

lr = 0.0001
opt = opt = tf.keras.optimizers.SGD(learning_rate=lr)
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError()
model = my_model_fully_connected(loss_fn, opt, ishape=len(X_train[0]), lr= lr)
print(model.summary())


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 27)]              0         
                                                                 
 dense_21 (Dense)            (None, 100)               2800      
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_22 (Dense)            (None, 50)                5050      
                                                                 
 dense_23 (Dense)            (None, 25)                1275      
                                                                 
 dense_24 (Dense)            (None, 1)                 26        
                                                                 
Total params: 9,151
Trainable params: 9,151
Non-trainable p

In [32]:
Y_train[0:20]

array([[  6],
       [  4],
       [  6],
       [122],
       [ 82],
       [167],
       [ 18],
       [  0],
       [  3],
       [  1],
       [  0],
       [  0],
       [  0],
       [  0],
       [ 29],
       [  0],
       [  6],
       [  0],
       [  1],
       [  0]])

In [31]:
# Call backs 
model_name_fcn = "First_draft.h5"
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 200)

monitor = tf.keras.callbacks.ModelCheckpoint(model_name_fcn, monitor='val_loss',\
                                             verbose=0, save_best_only=True,\
                                             save_weights_only=True,\
                                             mode='min')
# Learning rate schedule
def scheduler(epoch, lr):
    if epoch%10 == 0:
        lr = lr
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = 0)

In [28]:
# Train
model.fit(X_train, Y_train, epochs = 100, batch_size=200, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule], validation_data= (X_val, Y_val))

Epoch 1/100
100/100 [==============================] - 1s 5ms/step - loss: 1.1184 - accuracy: 0.7142 - val_loss: 1.1027 - val_accuracy: 0.7481 - lr: 0.0100
Epoch 2/100
100/100 [==============================] - 0s 4ms/step - loss: 1.1048 - accuracy: 0.7158 - val_loss: 1.0956 - val_accuracy: 0.7479 - lr: 0.0100
Epoch 3/100
100/100 [==============================] - 0s 4ms/step - loss: 1.1012 - accuracy: 0.7162 - val_loss: 1.0948 - val_accuracy: 0.7485 - lr: 0.0100
Epoch 4/100
100/100 [==============================] - 0s 4ms/step - loss: 1.0999 - accuracy: 0.7163 - val_loss: 1.0948 - val_accuracy: 0.7481 - lr: 0.0100
Epoch 5/100
100/100 [==============================] - 0s 4ms/step - loss: 1.0988 - accuracy: 0.7161 - val_loss: 1.0948 - val_accuracy: 0.7481 - lr: 0.0100
Epoch 6/100
100/100 [==============================] - 0s 4ms/step - loss: 1.0964 - accuracy: 0.7161 - val_loss: 1.0943 - val_accuracy: 0.7465 - lr: 0.0100
Epoch 7/100
100/100 [==============================] - 0s 4ms/st

In [89]:
print(str(len(X_train[0])))
[model.predict(X_train[3].reshape([1,27])), Y_train[0]]

27


[array([[0.]], dtype=float32), array([6])]

## Package DNN

### Load data

In [33]:
target_scaler = MinMaxScaler()
data = pkg_df
df = data.iloc[:,3:-1]
data_columns = df.columns
X_np = np.array(df)
target_scaler.fit(X_np)
X = target_scaler.transform(X_np)

Y = data.iloc[:,-1]
Y = Y.values.reshape(-1, 1)
target_scaler = MinMaxScaler()
target_scaler.fit(Y)
Y = target_scaler.transform(Y)


In [34]:
splitter = int(len(X) * .8)
X_train = X[:splitter]
Y_train = Y[:splitter]
X_val = X[splitter:]
Y_val = Y[splitter:]

### Train model

In [43]:
def my_model_fully_connected(loss_fn, opt, ishape = 27, lr = 1e-2):
    model_input = tf.keras.layers.Input(shape = ishape)
    l2 = tf.keras.layers.Dense(100, activation = 'sigmoid')(model_input)
    l2_drop = tf.keras.layers.Dropout(0.05)(l2)
    l3 = tf.keras.layers.Dense(50, activation = 'relu')(l2_drop)
    l4 = tf.keras.layers.Dense(25, activation = 'relu')(l3) 
    out= tf.keras.layers.Dense(1, activation = 'relu')(l4)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    model.compile(optimizer=opt, loss=loss_fn, metrics = ["accuracy"])
    return model

lr = 0.001
opt = opt = tf.keras.optimizers.SGD(learning_rate=lr)
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError()
model = my_model_fully_connected(loss_fn, opt, ishape=len(X_train[0]), lr= lr)
print(model.summary())


Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 18)]              0         
                                                                 
 dense_33 (Dense)            (None, 100)               1900      
                                                                 
 dropout_10 (Dropout)        (None, 100)               0         
                                                                 
 dense_34 (Dense)            (None, 50)                5050      
                                                                 
 dense_35 (Dense)            (None, 25)                1275      
                                                                 
 dense_36 (Dense)            (None, 1)                 26        
                                                                 
Total params: 8,251
Trainable params: 8,251
Non-trainable 

In [44]:
model_name_fcn = "package_draft.h5"
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 200)

monitor = tf.keras.callbacks.ModelCheckpoint(model_name_fcn, monitor='val_loss',\
                                             verbose=0, save_best_only=True,\
                                             save_weights_only=True,\
                                             mode='min')
# Learning rate schedule
def scheduler(epoch, lr):
    if epoch%10 == 0:
        lr = lr/2
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = 0)

In [45]:
## Train model
model.fit(X_train, Y_train, epochs = 100, batch_size=32, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule], validation_data= (X_val, Y_val))

Epoch 1/100
55/55 [==============================] - 1s 12ms/step - loss: 0.0238 - accuracy: 0.5251 - val_loss: 0.0109 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 2/100
55/55 [==============================] - 0s 5ms/step - loss: 0.0142 - accuracy: 0.5251 - val_loss: 0.0051 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 3/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0084 - accuracy: 0.5251 - val_loss: 0.0027 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 4/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0055 - accuracy: 0.5251 - val_loss: 0.0019 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 5/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0035 - accuracy: 0.5251 - val_loss: 0.0017 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 6/100
55/55 [==============================] - 0s 4ms/step - loss: 0.0028 - accuracy: 0.5251 - val_loss: 0.0017 - val_accuracy: 0.6082 - lr: 5.0000e-04
Epoch 7/100
55/55 [==============================] 

KeyboardInterrupt: ignored

In [ ]:
## Test model